<a href="https://colab.research.google.com/github/francotestori/aauba_02/blob/master/Armado_de_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabajo Practico 2

## Armado del dataset

In [1]:
!wget http://download.tensorflow.org/data/speech_commands_v0.01.tar.gz
!mkdir speechcommands
!tar -xf speech_commands_v0.01.tar.gz -C /content/speechcommands

'wget' is not recognized as an internal or external command,
operable program or batch file.
A subdirectory or file speechcommands already exists.
tar: Error opening archive: Failed to open 'speech_commands_v0.01.tar.gz'


In [2]:
# Import de Librerías
import librosa
import glob
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from IPython.display import Audio
from librosa.display import specshow

In [3]:
# Busco y guardo en un mapa todos los audio files correspondientes a los digit_keys
digit_command_keys = [
  'zero',                    
  'one',
  'two',
  'three',
  'four',
  'five',
  'six',
  'seven',
  'eight',
  'nine'
]



In [4]:
# Utilizamos la función propuesta en el TP 
# para hacer la extracción de los datos 
# correspondientes a cada audio file

def calculate_features(
        filename,
        n_mfcc=12,
        delta=True,
        deltadelta=True,
        energy=True, 
        summary_fn = [np.mean, np.std], 
        summary_names=['mean','std']):
    #Abro el archivo:
    x, sr = librosa.core.load(filename,sr=None)

    #Calculo MFCCs
    features = librosa.feature.mfcc(x,sr=sr,n_mfcc=n_mfcc)
    feat_names = ['mfcc_{}'.format(i) for i in range(n_mfcc)]
    #Calculo energia:
    if energy:
        energy = librosa.feature.rms(x)
        features = np.concatenate([features,energy])
        feat_names = feat_names + ['energy']
    #Aplico media y desvio estandar por defecto
    summary_features = np.concatenate([fn(features,axis=1) for fn in summary_fn])
    feat_names = ['{}_{}'.format(name_i,summ_i) for summ_i in summary_names for name_i in feat_names]

    #Lo mismo con los delta
    if delta:
        deltafeatures = np.diff(features)
        summary_features = np.concatenate([summary_features,np.concatenate([fn(deltafeatures,axis=1) for fn in summary_fn])])
        d_names = ['d{}'.format(name) for name in feat_names]
    else:
        d_names = []

    #Y con los delta de segundo orden
    if deltadelta:
        deltadeltafeatures = np.diff(features,n=2)
        summary_features = np.concatenate([summary_features,np.concatenate([fn(deltadeltafeatures,axis=1) for fn in summary_fn])]) 
        dd_names = ['dd{}'.format(name) for name in feat_names]
    else:
        dd_names = []

    feat_names = feat_names + d_names + dd_names

    return summary_features, feat_names  

In [10]:
# NOISE SOURCE

folders = [
  'sc_STREET-CITY-1_lvl_10',  
  'sc_CAFE-CAFE-1_lvl_10',
  'sc_CAFE-FOODCOURTB-1_lvl_10',
  'sc_HOME-KITCHEN-1_lvl_10',
]

folders = [
  'speechcommands_AWN_100_2', 
  'speechcommands_AWN_80_2', 
  'speechcommands_AWN_60_2', 
  'speechcommands_AWN_40_2', 
  'speechcommands_AWN_20_2',
]


folders = [
  'audio_digits_RUIDO_BLANCO_3',
  'audio_digits_RUIDO_BLANCO_10',
  'audio_digits_RUIDO_BLANCO_30',
  'audio_digits_RUIDO_BLANCO_100',
  'audio_digits_RUIDO_BLANCO_300',
  'audio_digits_RUIDO_BLANCO_1000',
  'audio_digits_RUIDO_BLANCO_3000',
]


In [9]:
for folder in folders:
    audio_files = {}
    for key in digit_command_keys:
        directory = f'../{folder}/{key}/*.wav'
        audio_files[key] = glob.glob(directory)
    print(folder)
    audio_df = pd.DataFrame()
    k=0
    for key in audio_files:
        print(key)
        j=0
        for audio in audio_files[key]:
            feat, names = calculate_features(audio)
            audio_row = dict(zip(names, feat))
            audio_row['digit'] = key
            audio_row['filename'] = audio
            audio_df = audio_df.append(audio_row, ignore_index=True)
            k+=1
            j+=1
        print(f'Procesados {j} audios en {folder}/{key}.')
    print(f'Procesados {k} audios en {folder}.')
    audio_df.to_csv(f'resources/{folder}.csv')

audio_digits_RUIDO_BLANCO_3
zero
Procesados 250 audios en audio_digits_RUIDO_BLANCO_3/zero.
one
Procesados 248 audios en audio_digits_RUIDO_BLANCO_3/one.
two
Procesados 264 audios en audio_digits_RUIDO_BLANCO_3/two.
three
Procesados 267 audios en audio_digits_RUIDO_BLANCO_3/three.
four
Procesados 253 audios en audio_digits_RUIDO_BLANCO_3/four.
five
Procesados 271 audios en audio_digits_RUIDO_BLANCO_3/five.
six
Procesados 244 audios en audio_digits_RUIDO_BLANCO_3/six.
seven
Procesados 239 audios en audio_digits_RUIDO_BLANCO_3/seven.
eight
Procesados 257 audios en audio_digits_RUIDO_BLANCO_3/eight.
nine
Procesados 259 audios en audio_digits_RUIDO_BLANCO_3/nine.
Procesados 2552 audios en audio_digits_RUIDO_BLANCO_3.
